Basic definitions

In [47]:
import numpy as np
up = np.array([1., 0.])
down = np.array([0., 1.])
updown = np.kron(up, down)
downup = np.kron(down, up)
upup = np.kron(up, up)
downdown = np.kron(down, down)
state = np.kron(up, up)

CX = np.array([[1., 0., 0., 0.,], [0., 1., 0., 0.], [0., 0., 0., 1.], [0., 0., 1., 0.]])
I = np.eye(2)
X = np.array([[0., 1.], [1., 0.]])
Y = np.array([[0., -1j], [1j, 0.]])
Z = np.array([[1., 0.], [0, -1]])
H = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])

In [2]:
number_of_qubits = 2 # find better way of reading this from the state or circuit
circ = QuantumCircuit(number_of_qubits)
circ.x([0])
circ.cx(0, 1)
circ.x([1])

print(circ.draw())

ops = []

def apply_circuit(state, circuit):
    ops = []
    for index, instruction in enumerate(circ):
        # index is the gate number in the total number of gates. 
        # instruction consists of something like (operation, qubit) with operation as (name='x', num_qubits=1, num_clbits=0, params=[])
        
        # Operator converts this operation into a matrix. 
        gate = Operator(instruction.operation) 
        if instruction.operation.name == 'cx':
            gate = CX # since they go from MSB to LSB, their cx is unusual. maybe there's a more elegant solution though.
        ops.append(instruction.operation.name)

        # get the index of the qubits applied
        qubit_indices = []
        num_qubits = instruction.operation._num_qubits
        n = 0

        # save index numbers in array
        while n < num_qubits:
            index = instruction.qubits[n]._index
            qubit_indices.append(index)
            # assume adjacent CNOTs for now; eventually add the exception for non-adjacent CNOTs
            n = n + 1
        final_matrix = kron_products(gate, qubit_indices, number_of_qubits)
        state = np.dot(final_matrix, state)
        print(state)
        print("mat = ", final_matrix)
    print(ops)
    return state



def apply_tensorized_circ(state, circuit):
    

NameError: name 'QuantumCircuit' is not defined

In [4]:
import numpy as np
def kron_products(gate, indices, number_of_qubits):
    # this assumes we are given some fixed gate, with an array of indices this gate is applied to. 
    # For now, assume 2-qubit gates that are adjacent and increasing in order (that is, CNOT(2, 3), not CNOT(3, 2)). I assume that if this isn't the case, we will add SWAP gates to make this happen. 

    I = np.eye(2)

    start_index = 1
    if indices[0] == 0:
        op = gate
        if len(indices) == 2: # for two-qubit gates, start one later
            start_index = 2
    else:
        op = I

    marker = 0 # used to skip following loops for two-qubit gates 
    for n in range(start_index, number_of_qubits):
        if marker == 1:
            marker = 0
            continue
        if n in indices:
            op = np.kron(op, gate)
            if len(indices) == 2:
                marker = 1
        else:
            op = np.kron(op, I)
        
    return op

# Tensorized version of the circuit

Functions for the tensorized circuit

In [28]:


import numpy as np

# standard definitions
up = np.array([1., 0.])
down = np.array([0., 1.])
state = np.kron(up, up)

# more standard state definitions for testing
updown = np.kron(up, down)
downup = np.kron(down, up)
upup = np.kron(up, up)
downdown = np.kron(down, down)

# gate definitions
CX = np.array([[1., 0., 0., 0.,], [0., 1., 0., 0.], [0., 0., 0., 1.], [0., 0., 1., 0.]])
I = np.eye(2)
X = np.array([[0., 1.], [1., 0.]])
Y = np.array([[0., -1j], [1j, 0.]])
Z = np.array([[1., 0.], [0, -1]])
H = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])

# functions

def apply_1qubit(psi, j, op):
    """
    psi: np.array (2, ..., 2)
    j: integer between 0 and L-1
    op: np array (2,2)
    """
    # print('size of op is', op.shape)
    # print('size of wavefct is', psi.shape)
    L = len(psi.shape)
    psi = np.tensordot(op, psi, axes = ([1], [j])) # j [j*], i1 i2 ... [j] ... iL
    t = [i + 1 for i in range(j)] + [0] + [i for i in range(j+1, L)]
    psi = np.transpose(psi, t)
    return psi

def apply_2qubit(psi, op, control, target):
    """
    psi: np.array (2, ..., 2)
    j: integer between 0 and L-2 (?)
    op: np array (4, 4)
    """
    # CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]).reshape((2,2,2,2))
    op = np.reshape(op, (2, 2, 2, 2))
    psi = np.tensordot(op, psi, axes = ([0, 1], [control, target]))
    psi = np.moveaxis(psi, (0, 1), (control, target)) # for two qubits?
    return psi


def reshape_state(state):
    s = np.size(state)
    if (s % 2 != 0):
        print(0)
    N = int(s / 2)
    return state.reshape((2,)* N)

def init_spinup(L):
    psi = np.zeros(2**L)
    psi[0] = 1
    psi = psi.reshape([2 for i in range(L)])
    return psi

Tensorize things

In [37]:
from qiskit.quantum_info import Operator

def apply_tensorized_circuit(state, circuit):
    # ops = []
    for index, instruction in enumerate(circuit):
        # index is the gate number in the total number of gates. 
        # instruction consists of something like (operation, qubit) with operation as (name='x', num_qubits=1, num_clbits=0, params=[])
        
        # Operator converts this operation into a matrix. 
        gate = np.array(Operator(instruction.operation))
    
        if instruction.operation.name == 'cx':
            gate = CX # since they go from MSB to LSB, their cx is unusual. maybe there's a more elegant solution though.
        # ops.append(instruction.operation.name)
        # ops.append(gate)

        # get the index of the qubits applied locally
        qubit_indices = []
        num_qubits = instruction.operation._num_qubits
        n = 0

        # save index numbers in array
        while n < num_qubits:
            index = instruction.qubits[n]._index
            qubit_indices.append(index)
            # assume adjacent CNOTs for now; eventually add the exception for non-adjacent CNOTs
            n = n + 1
        state = reshape_state(state)
        if num_qubits not in [1, 2]:
            print('we only support 1 and 2 qubit gates for now.')
            break
        if num_qubits == 1:
            print('operator shape is', gate.shape)

            print('psi shape is', state.shape)
            state = apply_1qubit(psi=state, j=qubit_indices[0], op=gate)
        elif num_qubits == 2:
            state = apply_2qubit(psi=state, op=gate, control=qubit_indices[0], target=qubit_indices[1]) # assume that the control and target are also provided in order in qiskit
        
        # final_matrix = kron_products(gate, qubit_indices, number_of_qubits)
        # state = np.dot(final_matrix, state)
        # print(state)
        # print("mat = ", final_matrix)
    # print(ops)
    # print(qubit_indices)
    return state


# CLEAN VERSION:
from qiskit.quantum_info import Operator

def apply_tensorized_circuit(state, circuit):
    
    for index, instruction in enumerate(circuit):
        # index is the gate number in the total number of gates. 
        # instruction consists of something like (operation, qubit) with operation as (name='x', num_qubits=1, num_clbits=0, params=[])
        
        # Operator converts this operation into a matrix. 
        gate = np.array(Operator(instruction.operation))
    
        if instruction.operation.name == 'cx':
            gate = CX # since they go from MSB to LSB, their cx is unusual. maybe there's a more elegant solution though.

        # get the index of the qubits applied locally
        qubit_indices = []
        num_qubits = instruction.operation._num_qubits
        n = 0

        # save index numbers in array
        while n < num_qubits:
            index = instruction.qubits[n]._index
            qubit_indices.append(index)
            # assume adjacent CNOTs for now; eventually add the exception for non-adjacent CNOTs
            n = n + 1
        state = reshape_state(state)
        # we only look at 1, 2 qubit gates
        if num_qubits not in [1, 2]:
            print('we only support 1 and 2 qubit gates for now.')
            break
        if num_qubits == 1:
            print('operator shape is', gate.shape)

            print('psi shape is', state.shape)
            state = apply_1qubit(psi=state, j=qubit_indices[0], op=gate)
        elif num_qubits == 2:
            state = apply_2qubit(psi=state, op=gate, control=qubit_indices[0], target=qubit_indices[1]) # assume that the control and target are also provided in order in qiskit
        
    return state



In [48]:
from qiskit import QuantumCircuit

L = 4
c = QuantumCircuit(L)
psi = upup
should = downup
c.x(0)
# c.h(1)
c.cx(0, 1)
c.x(1)
psi2 = apply_tensorized_circuit(psi, c)
print('our state is \n', psi2, '\n the expected value is \n ', reshape_state(should))
# np.array_equal(should, psi2)

operator shape is (2, 2)
psi shape is (2, 2)
operator shape is (2, 2)
psi shape is (2, 2)
our state is 
 [[0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j]] 
 the expected value is 
  [[0. 0.]
 [1. 0.]]


Add noise

In [4]:
# define kraus operators

import numpy as np
g1 = 1 # 1/T1
# p = 1 - np.exp(-g1*dt)
# np.random.rand()
# for t in ts:
    if np.random.rand() < 

0.15089511071630346

Old code scraps

In [7]:
# import numpy as np
# # take a state that is 2^N and divide into N sites


# state = np.kron(np.kron(up, up), np.kron(up, up))
# s = np.size(state)
# if (s % 2 != 0):
#     print(0)
# N = int(s / 2)
# state = np.reshape(state, (N, 2))
# print(state)
# print(np.einsum(state, X, axes=([1], [0])))

[[1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


TypeError: each subscript must be either an integer or an ellipsis

In [11]:
import numpy as np

up = np.array([1., 0.])
down = np.array([0., 1.])
state = np.kron(up, up)


CX = np.array([[1., 0., 0., 0.,], [0., 1., 0., 0.], [0., 0., 0., 1.], [0., 0., 1., 0.]])
I = np.eye(2)
X = np.array([[0., 1.], [1., 0.]])
Y = np.array([[0., -1j], [1j, 0.]])
Z = np.array([[1., 0.], [0, -1]])
H = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])

print(X.shape)


M = np.kron(up, down)
M
s = np.size(M)
if (s % 2 != 0):
    print(0)
N = int(s / 2)
M = np.reshape(M, (N, 2))
print(M)
('my result is', np.einsum('j..., ij', M, X))
('the expected result is', np.reshape(np.kron(down, down), (N, 2)))

def reshape_state(state):
    s = np.size(state)
    if (s % 2 != 0):
        print(0)
    N = int(s / 2)
    M = np.reshape(state, (N, 2))

(2, 2)
[[0. 1.]
 [0. 0.]]


In [43]:
# from an old project
import numpy as np
# take a state that is 2^N and divide into N sites
updown = np.kron(up, down)
downup = np.kron(down, up)
upup = np.kron(up, up)
downdown = np.kron(down, down)

# state = np.kron(np.kron(up, up), np.kron(up, up))
# s = np.size(state)
# if (s % 2 != 0):
#     print(0)
# N = int(s / 2)
# state = np.reshape(state, (N, 2))
# print(state)
# print(np.einsum(state, X, axes=([1], [0])))

# L = 8 # number of sites, or number of qubits


def apply_1qubit(psi, j, op):
    """
    psi: np.array (2, ..., 2)
    j: integer between 0 and L-1
    op: np array (2,2)
    """
    # print('size of op is', op.shape)
    # print('size of wavefct is', psi.shape)
    L = len(psi.shape)
    psi = np.tensordot(op, psi, axes = ([1], [j])) # j [j*], i1 i2 ... [j] ... iL
    t = [i + 1 for i in range(j)] + [0] + [i for i in range(j+1, L)]
    psi = np.transpose(psi, t)
    return psi

def apply_2qubit(psi, op, control, target):
    """
    psi: np.array (2, ..., 2)
    j: integer between 0 and L-2 (?)
    op: np array (4, 4)
    """
    # CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]).reshape((2,2,2,2))
    op = np.reshape(op, (2, 2, 2, 2))
    psi = np.tensordot(op, psi, axes = ([0, 1], [control, target]))
    psi = np.moveaxis(psi, (0, 1), (control, target)) # for two qubits?
    return psi


def reshape_state(state):
    s = np.size(state)
    if (s % 2 != 0):
        print(0)
    N = int(s / 2)
    return state.reshape((2,)* N)

def init_spinup(L):
    psi = np.zeros(2**L)
    psi[0] = 1
    psi = psi.reshape([2 for i in range(L)])
    return psi

# psi = init_spinup()
# test one qubit gate
M = reshape_state(downdown)
# print(len(psi.shape))
# print(psi)
M = apply_2qubit(M, CX, control=0, target=1)

should = reshape_state(downup)
np.array_equal (M, should)
print('it is \n', M, '\n it should be \n', should)
# print(apply_1qubit(M, 0, X, L))
# print(np.reshape(np.kron(down, down), (2, N)))
# apply_2qubit(M, CX, 0, 1)


it is 
 [[0. 0.]
 [1. 0.]] 
 it should be 
 [[0. 0.]
 [1. 0.]]


Other other code scraps

In [76]:
# import numpy as np

# # Define basic states and gates
# up = np.array([1., 0.])
# down = np.array([0., 1.])
# X = np.array([[0., 1.], [1., 0.]])

# def apply_1qubit(psi, j, op):
#     """
#     Apply a single qubit operation to a multi-qubit state.

#     psi: The state as an array of shape (2,) * N
#     j: The qubit index to which the gate is applied
#     op: The operation matrix (2x2)
#     """
#     # Get the number of qubits based on the shape of the state
#     num_qubits = len(psi.shape)
    
#     # Prepare an identity matrix for every other qubit
#     identities = [np.eye(2) for _ in range(num_qubits)]
    
#     # Replace the identity matrix at index j with the operation
#     identities[j] = op
    
#     # Compute the tensor product of all these matrices
#     # This will give us a tensor that represents the operator expanded to all qubits
#     operator = identities[0]
#     for i in range(1, num_qubits):
#         operator = np.kron(operator, identities[i])

#     # Reshape the state to a vector for matrix multiplication
#     psi_vector = psi.reshape(-1)
    
#     # Apply the expanded operator to the state vector
#     result_vector = np.dot(operator, psi_vector)
    
#     # Reshape the result back to the original state shape
#     return result_vector.reshape(psi.shape)

# def reshape_state(state, qubits):
#     """
#     Reshape a state vector into a multi-dimensional array for individual qubit manipulation.
#     """
#     return state.reshape((2,) * qubits)

# # Example: Creating a 2-qubit state |00⟩
# state = np.kron(down, down)

# # Reshape state for multi-qubit operations
# state = reshape_state(state, 2)

# # Apply an X gate to the first qubit (qubit 0)
# modified_state = apply_1qubit(state, 0, X)

# # Check results
# expected_state = reshape_state(np.kron(up, down), 2)
# print("Modified state:\n", modified_state)
# print("Expected state:\n", expected_state)
# print('Is the transformation correct?', np.array_equal(modified_state, expected_state))


Modified state:
 [[0. 1.]
 [0. 0.]]
Expected state:
 [[0. 1.]
 [0. 0.]]
Is the transformation correct? True
